In [1]:
import models.utils
import random, gc, os, pickle, csv, time, re
import numpy as np

In [2]:
# csv_path = "/data/model_runs/original_oml/aOML-order1-inlr010-2022-08-30-sr-query/OML-order1-id4-2022-08-30_05-21-18.854228_update50_results_sr_ta.csv"
csv_path = "/data/model_runs/original_oml/aOML-order1-inlr010-2022-08-30-sr-query/OML-order1-id4-2022-08-30_05-21-18.854228_update50_results_sr_ta2.csv"
csv_path = "/data/model_runs/original_oml/aOML-order1-inlr010-2022-08-30-sr-query/OML-order1-id4-2022-08-30_05-21-18.854228_update5_results_sr_ta_ss.csv"
csv_path = "/data/model_runs/original_oml/aOML-order1-inlr010-2022-08-30-sr-query/OML-order1-id4-2022-08-30_05-21-18.854228_update5_results_sr_ta_ss2.csv"
csv_path = "/data/model_runs/original_oml/aOML-order1-inlr010-2022-08-30-sr-query/OML-order1-id4-2022-08-30_05-21-18.854228_update5_results_sr_ta_ss3.csv"
csv_path = "/data/model_runs/original_oml/aOML-order1-inlr010-2022-08-30-sr-query/OML-order1-id4-2022-08-30_05-21-18.854228_update5_results_sr_ta_ss4.csv"
csv_path ="/data/model_runs/original_oml/aOML-order1-inlr010-2022-08-30-sr-query/OML-order1-id4-2022-08-30_05-21-18.854228_update5_results_sr_ta_ss5.csv"
csv_path ="/data/model_runs/original_oml/aOML-order1-inlr010-2022-08-30-sr-query/OML-order1-id4-2022-08-30_05-21-18.854228_update5_results_sr_ta_ss6.csv"
csv_path = "/data/model_runs/original_oml/aOML-order1-inlr010-2022-08-30-sr-query/OML-order1-id4-2022-08-30_05-21-18.854228_update5_results_sr_ta_ss7.csv"
csv_path = "/data/model_runs/original_oml/aOML-order1-inlr010-2022-08-30-sr-query/OML-order1-id4-2022-08-30_05-21-18.854228_update5_results_sr_ta_ss8.csv"
csv_path = "/data/model_runs/original_oml/aOML-order1-inlr010-2022-14-21-sreversed/OML-order1-id4-2022-12-15_03-55-41.428520_update5_results.csv"

csv_path = "/data/model_runs/original_oml/aOML-order1-inlr010-2023-01-20-srev-replay2/OML-order1-id4-2023-01-21_19-49-04.497477_update5_results_nosort.csv"
csv_path = "/data/model_runs/original_oml/aOML-order1-inlr010-2023-02-13-srev-curri/OML-order1-id4-2023-02-13_05-46-13.147873_update5_results.csv"

csv_path = "/data/model_runs/original_oml/aOML-order4-inlr005-2023-02-16-srev-curri/OML-order4-id4-2023-02-17_00-51-13.202492_update10_results.csv"
csv_path = "/data/model_runs/original_oml/aOML-order3-inlr005-2023-02-16-srev-curri/OML-order3-id4-2023-02-16_14-53-18.742272_update10_results.csv"
csv_path = "/data/model_runs/original_oml/aOML-order2-inlr005-2023-02-16-srev-curri/OML-order2-id4-2023-02-16_01-45-47.923694_update10_results.csv"
csv_path = "/data/model_runs/original_oml/aOML-order1-inlr005-2023-02-14-srev-curri/OML-order1-id4-2023-02-14_04-13-46.616300_update10_results.csv"
csv_path = "/data/model_runs/original_oml/aOML-order1-inlr005-2023-03-07-pffeifer/OML-order1-id4-2023-03-07_04-05-29.829256_update5_results.csv"
csv_path = "/data/model_runs/original_oml/aOML-order1-inlr005-2023-03-07-lorav11/OML-order1-id4-2023-03-07_22-12-41.477539_update5_results.csv"
csv_path = "/data/model_runs/original_oml/aOML-order4-2023-03-13-no12/OML-order4-id4-2023-03-13_02-29-39.653929_update5_results.csv"
csv_path = "/data/model_runs/original_oml/aOML-order3-2023-03-14-v12/OML-order3-id4-2023-03-13_22-50-18.310019_update5_results.csv"


BASE_MODEL_PATH = "/data/model_runs/original_oml/aOML-order1-inlr010-2022-07-31"
find_path = "id4.*update5_"
csv_name = [x for x in os.listdir(BASE_MODEL_PATH) if  re.search(f"{find_path}.*results\.csv$", x)][0]
csv_path = os.path.join(BASE_MODEL_PATH, csv_name)
print(f"Using csv from {csv_path}")


Using csv from /data/model_runs/original_oml/aOML-order1-inlr010-2022-07-31/OML-order1-id4-2022-07-31_21-18-36.241546_update5_results.csv


In [3]:
# Map of dataset -> Array<number>
label_dict, pred_dict = {}, {}
with open(csv_path) as csv_file:
    csv_reader = csv.reader(csv_file, delimiter=',')
    line_count = 0
    for row in csv_reader:
        if line_count == 0:
            line_count += 1
            continue
        #print(row)
        
        dataset = ''.join([i for i in row[0] if not i.isdigit()]) # Remove all numbers to get dataset
        label = int(row[1])
        pred = int(row[3])
        
        label_dict[dataset] = label_dict.get(dataset, []) + [label]
        pred_dict[dataset]  = pred_dict.get(dataset, []) + [pred]

        line_count += 1

print(label_dict.keys())

dict_keys(['YelpDataset', 'AGNewsDataset', 'DBPediaDataset', 'AmazonDataset', 'YahooAnswersDataset'])


In [4]:
keys = ['YelpDataset', 'AGNewsDataset', 'DBPediaDataset', 'AmazonDataset', 'YahooAnswersDataset']

accuracies, precisions, recalls, f1s = [], [], [], []
for key in keys:
    acc, prec, rec, f1 = models.utils.calculate_metrics(pred_dict[key], label_dict[key])
    accuracies.append(acc)
    precisions.append(prec)
    recalls.append(rec)
    f1s.append(f1)
    
print()
print("COPY PASTA - not really but ok")
for row in accuracies:
    print(row)
print()
print('Overall test metrics: Accuracy = {:.4f}, precision = {:.4f}, recall = {:.4f}, '
            'F1 score = {:.4f}'.format(np.mean(accuracies), np.mean(precisions), np.mean(recalls), np.mean(f1s)))



COPY PASTA - not really but ok
0.5592105263157895
0.8971052631578947
0.9860526315789474
0.5634210526315789
0.7544736842105263

Overall test metrics: Accuracy = 0.7521, precision = 0.7725, recall = 0.7529, F1 score = 0.7501
